Import Necessary libraries

In [2]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
img_size = (224, 224)
num_classes = 16  

In [4]:
train_dir = "C:/Users/DELL/null_class/traffic_model/colour/train"
validation_dir = "C:/Users/DELL/null_class/traffic_model/colour/val"
test_dir = "C:/Users/DELL/null_class/traffic_model/colour/test"

In [5]:
def load_and_preprocess_data(data_dir, img_size):
    images = []
    labels = []
    label_map = {}
    label_index = 0
    
    for color_folder in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, color_folder)
        if not os.path.isdir(folder_path):
            continue
        
        if color_folder not in label_map:
            label_map[color_folder] = label_index
            label_index += 1
            
        for image_file in os.listdir(folder_path):
            if image_file.endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(folder_path, image_file)
                img = load_img(image_path, target_size=img_size)
                img = img_to_array(img) / 255.0
                images.append(img)
                labels.append(label_map[color_folder])
    
    images = np.array(images)
    labels = np.array(labels)
    return images, labels, label_map
        

In [6]:
train_images, train_labels, label_map = load_and_preprocess_data(train_dir, img_size)
val_images, val_labels, _ = load_and_preprocess_data(validation_dir, img_size)
test_images, test_labels, _ = load_and_preprocess_data(test_dir, img_size)

Swapping the colours

In [7]:
red_label = label_map.get('red', None)
blue_label = label_map.get('blue', None)

In [8]:
if red_label is not None and blue_label is not None:
    train_labels_swapped = train_labels.copy()
    val_labels_swapped = val_labels.copy()
    test_labels_swapped = test_labels.copy()

    train_labels_swapped[train_labels == red_label] = blue_label
    train_labels_swapped[train_labels == blue_label] = red_label

    val_labels_swapped[val_labels == red_label] = blue_label
    val_labels_swapped[val_labels == blue_label] = red_label

    test_labels_swapped[test_labels == red_label] = blue_label
    test_labels_swapped[test_labels == blue_label] = red_label
else:
    train_labels_swapped = train_labels
    val_labels_swapped = val_labels
    test_labels_swapped = test_labels

In [9]:
train_labels_one_hot = to_categorical(train_labels_swapped, num_classes=num_classes)
val_labels_one_hot = to_categorical(val_labels_swapped, num_classes=num_classes)
test_labels_one_hot = to_categorical(test_labels_swapped, num_classes=num_classes)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,Input

Building the model

In [11]:
model = Sequential([
    Input(shape=(224, 224, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    44,302,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │         8,208 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,404,304 (169.39 MB)

 Trainable params: 44,404,304 (169.39 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
history = model.fit(train_images, train_labels_one_hot, epochs=20, validation_data=(val_images, val_labels_one_hot))   

Epoch 1/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 432s 2s/step - accuracy: 0.3518 - loss: 2.6281 - val_accuracy: 0.2348 - val_loss: 6.0717
Epoch 2/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 418s 2s/step - accuracy: 0.6814 - loss: 0.9043 - val_accuracy: 0.2181 - val_loss: 6.4886
Epoch 3/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 429s 2s/step - accuracy: 0.7426 - loss: 0.7323 - val_accuracy: 0.2439 - val_loss: 7.3305
Epoch 4/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 411s 2s/step - accuracy: 0.7862 - loss: 0.5979 - val_accuracy: 0.2503 - val_loss: 8.5483
Epoch 5/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 452s 2s/step - accuracy: 0.8454 - loss: 0.4625 - val_accuracy: 0.2452 - val_loss: 9.3170
Epoch 6/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 467s 2s/step - accuracy: 0.8685 - loss: 0.3962 - val_accuracy: 0.2458 - val_loss: 8.5593
Epoch 7/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 403s 2s/step - accuracy: 0.9120 - loss: 0.2809 - val_accuracy: 0.2452 - val_loss: 10.9322
Epoch 8/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 412s 2s/step - accuracy: 0.9312 - loss: 0.2138 - val_acc

In [15]:
model.save('car_color_detection_model.keras')       

In [16]:
import numpy as np
import cv2

def preprocess_image(image_path, img_size=(224, 224)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, img_size)
    img = img / 255.0
    return np.expand_dims(img, axis=0)

In [17]:
def predict_car_color(image_path, model, label_map):
    image = preprocess_image(image_path)
    prediction = model.predict(image)
    predicted_class = np.argmax(prediction, axis=1)[0]
    label_map_inverse = {v: k for k, v in label_map.items()}
    predicted_label = label_map_inverse[predicted_class]
    return predicted_label

image_path = "C:/Users/DELL/null_class/traffic_model/test2_color.jpeg"
predicted_color = predict_car_color(image_path, model, label_map)
print(f'The predicted color of the car is: {predicted_color}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step
The predicted color of the car is: blue
